In [ ]:
#dependencies
from pathlib import Path
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd
from datetime import datetime


Main_csv_path = Path("Resources/18100205.csv")
Metadata_csv_path = Path("Resources/18100205_MetaData.csv")

Main_StatsCan_IndexDF = pd.read_csv(Main_csv_path, encoding="UTF-8-Sig")
Metadata_DF = pd.read_csv(Metadata_csv_path, encoding="UTF-8-Sig")

Main_StatsCan_IndexDF.tail()

In [ ]:
Metadata_DF.head()

In [ ]:
# Select necessary columns for initial data cleaning processes.

reduced_main_df = Main_StatsCan_IndexDF[["REF_DATE","GEO","New housing price indexes","UOM","VALUE","STATUS","DECIMALS"]]		
#reduced_main_df.head()

reduced_main_df.columns.values[2] = "NewHousePrice_Index_Type" #Replaced column name of "New housing price indexes".

reduced_main_df.tail()

In [ ]:
#import re
#print(re.split(r',',reduced_main_df["GEO"]))
Orig_Geo_series = pd.Series(reduced_main_df["GEO"])
#print(Orig_Geo_series)
Geo_Split = Orig_Geo_series.str.split(',')

Geo_CityRow = Geo_Split.str.get(0) #access the first element for eventual "column1"

Geo_CityRow

In [ ]:
Geo_ProvRow = Geo_Split.str.get(1) #access the first element for eventual "column2"

Geo_ProvRow

In [ ]:
#Temp_DF = pd.concat(reduced_main_df,Geo_City )

Manual_Merge_Geo = pd.concat([Geo_CityRow,Geo_ProvRow], axis=1) #axis=1 is using "column 1" to merge.

Manual_Merge_Geo

#Rename Options presented via => https://www.geeksforgeeks.org/rename-specific-columns-in-pandas/
Manual_Merge_Geo.columns.values[0] = "City_Region"
Manual_Merge_Geo.columns.values[1] = "Province"

Manual_Merge_Geo

#BEST PRACTICE MIGHT BE TO RENAME RIGHT AWAY "HERE" ... to save coding time later.

In [ ]:
Orig_Year_series = pd.Series(reduced_main_df["REF_DATE"])
Year_Split = Orig_Year_series.str.split('-')

Row_Year = Year_Split.str.get(0) #access the first element for eventual "column1"

Row_Year

In [ ]:
Row_Month = Year_Split.str.get(1) #access the first element for eventual "column1"

Row_Month

In [ ]:
Manual_Merge_date = pd.concat([Row_Year,Row_Month], axis=1) #axis=1 is using "column 1" to merge.

Manual_Merge_date

#Rename Options presented via => https://www.geeksforgeeks.org/rename-specific-columns-in-pandas/
Manual_Merge_date.columns.values[0] = "Year"
Manual_Merge_date.columns.values[1] = "Month"

Manual_Merge_date
#BEST PRACTICE MIGHT BE TO RENAME RIGHT AWAY "HERE" ... to save coding time later.
#KEY REMINDER ... if we need the month to be transformed into "1" intstead of "01" (IE: INT from TEXT/VARCHAR) ... here is probably the best place as well.

In [ ]:
#via => https://appdividend.com/2020/06/02/pandas-concat-method-example-in-python/
#JOIN Split GEO Columns to Original DF

GeoUpdated_DF = pd.concat([reduced_main_df,Manual_Merge_Geo],axis=1,join='outer') # ,join_axes=None,ignore_index=False)

GeoUpdated_DF

#JOIN Split YEar+Month Columns to GeoUpdated_DF

DateUpdated_DF = pd.concat([GeoUpdated_DF,Manual_Merge_date],axis=1,join='outer')

DateUpdated_DF


In [ ]:
#reduced_main_df = Main_StatsCan_IndexDF[["REF_DATE","GEO","New housing price indexes","UOM","VALUE","STATUS","DECIMALS"]]

Analysis_DF = DateUpdated_DF[["Year","Month","City_Region","Province","NewHousePrice_Index_Type","UOM","VALUE","STATUS","DECIMALS"]]

Analysis_DF

In [ ]:
#Remove "junk data" columns based on StatsCan Legend.

StatusClean_Analysis_DF = Analysis_DF.query("STATUS not in ['E', 'x', '..']") # QUERY fuction delivers to python "memory" a "true/false" outcome.
TotalPriceIndex_Analysis_DF = StatusClean_Analysis_DF.query("NewHousePrice_Index_Type not in ['House only','']")
#StatusClean_Analysis_DF

TotalPriceIndex_Analysis_DF

In [ ]:
desired_cities = ['Toronto', 'Vancouver', 'Montréal']
filtered_city_data = {}

for city in desired_cities:
    city_filtered_df = TotalPriceIndex_Analysis_DF[TotalPriceIndex_Analysis_DF['City_Region'] == city]
    filtered_city_data[city] = city_filtered_df

# Print the data for all cities
for city, city_df in filtered_city_data.items():
    print(f"Data for {city}:")
    print(city_df)
    print("\n")

#Q from Ajani - 2023-07-24 - Are we confident that this is brining us all of the cities or is this the issue we discussed to look into validating? The validation was whether the code is producing all "cities" vs just the "last/first" one it finds in the list above.

#Solution1 Idea - 2023-07-24 - a NumPy Array may be necessary rather than a list. **OR** we create a Dictionary from the list ... with the Key being "City_Region" ... similar to our custom column rename above.


In [ ]:
#Extract sample data to validate that the STATUS not in funciton above was successful.
Output_Folder = 'Output/'
Output_file = Output_Folder + 'city_filtered_df.csv'

# Concatenate DataFrames for all cities
combined_data = pd.concat(filtered_city_data.values(), ignore_index=True)

#Q from Ajani ... we should look into this. It doesn't make sense for ignore_index=True here in my opinion. Very open to discussion. AO - 2023-07-24
#Q from Ajani ... are we sure all 3 cities are produced? Wondering if there's an iteration issue for Python/Pandas. Let's discuss at some point. AO - 2023-07-24

# Save the combined data to a single CSV file
combined_data.to_csv(Output_file, index=False, encoding='utf-8-sig') # ref for variation in export encoding https://stackoverflow.com/questions/57152985/what-is-the-difference-between-utf-8-and-utf-8-sig


In [ ]:
Housing_Prices_by_year = Path("Resources/Housing_Prices_by_year.csv")
Housing_Prices_by_year_df = pd.read_csv(Housing_Prices_by_year, encoding="UTF-8-Sig")

Housing_Prices_by_year_df

In [ ]:
#Validating which DataTypes are within the .csv of our new Housing Price DataSet.

Housing_Prices_by_year_df.dtypes

In [ ]:
#Rename columns for code error appearing associated to "Vancouver" (Likely a "text trim")  issue.

Housing_Prices_by_year_df.columns.values[1] = "Vancouver"
Housing_Prices_by_year_df.columns.values[2] = "Toronto"
Housing_Prices_by_year_df.columns.values[3] = "Montreal"

print(Housing_Prices_by_year_df)


In [ ]:
#Validating the Structure of the data from the .csv as Date Format input from "user" (the csv data set) to Python's/Panda's processing needs.
#Panda's processing needs can be observed here via URL => 

Housing_Prices_by_year_df['month_Year']

In [ ]:
#1 => Goal of next step is to "establish" a column being similar to StatsCan original data of "YYYY-MM".
#2 => We will use the following URL to establish what our "format=" parameter code has to be ... based on OUR USER INPUT DATA (IE: the .csv.) URL => https://www.w3schools.com/python/python_datetime.asp

pd.to_datetime(Housing_Prices_by_year_df['month_Year'], format='%Y-%m-%d', errors = 'coerce')

#SUCCESS ... (comment on the "why" to the team ... we are being "literal coders here" ... to produce the "date as a parameter" ... the params are "Y/M/D" ... and Python does it using the "to_datetime functions ... parameter that is called "format=" ... format='%Y-%m-%d' )
#For the "pd.to_datetime function ... the params are as follows.... (1 = you column to be processed/iterated over ... 2 = the format WITHIN that column ... 3 = what to do when the column you've targeted "doesn't fit" ... int the FORMAT you confirmed is the "data structure" for all entries. IE: basically we've developed an "#N/A" type output similar to an Excel "vlookup" ... but for dates in PYTHON ....)"

In [ ]:
#Next we have a "four stepper" as per our initial work above.
# This block is dedicated to "step #1" of repeating the pd.concat() approach above.
# pd.series() is CRITICAL for our "expected" output and data frame to "join to" ... the original ".csv" for us to then delete the "bad data" ... 
#
#Step #1 - 1> Orig_Geo_series = pd.Series(reduced_main_df["GEO"]) ... sets up "index" to match original dataset.
#Step #2 - 1> Geo_Split = Orig_Geo_series.str.split(',') ... splits up the "data" into our desired output.
#2 - 2> Geo_CityRow = Geo_Split.str.get(0) #access the first element for eventual "column1" ... sets up "variables" ... for renaming.
#2 - 3> Geo_CityRow ... Call's the row for "validation" ... 
#Step #3 - 1> DateUpdated_DF = pd.concat([GeoUpdated_DF,Manual_Merge_date],axis=1,join='outer')
#3 - 2> DateUpdated_DF ... Call's "merged data" for "validation" ...
#Step 4 - 1> #Manual_Merge_date.columns.values[0] = "Year" ... Renames the column outputs merged into new DF ... to "desired for presentation" naming convention.

#Pseudo code "Step 1" ... break the "user data" (the .csv) column into 3 parameters ...

Orig_HP_DateTime_To_Date = pd.Series(
     pd.to_datetime(Housing_Prices_by_year_df['month_Year'], format='%Y-%m-%d', errors = 'coerce').dt.strftime('%Y-%m-%d')
)

Orig_HP_DateTime_To_Date 


In [ ]:
#This cell is executing Step #2 above.

HP_Month_Year_DF = Orig_HP_DateTime_To_Date.str.split('-')

HP_Year_Row = HP_Month_Year_DF.str.get(0) #access the first element for eventual "column1"
HP_Month_Row = HP_Month_Year_DF.str.get(1) #access the 2nd element for eventual "column2"
HP_Day_Row = HP_Month_Year_DF.str.get(2) #access the 3rd element for eventual "column3"

print(HP_Year_Row)
print(HP_Month_Row)
print(HP_Day_Row)


#Orig_HP_DateSeries = pd.Series(
#    pd.to_datetime(Housing_Prices_by_year_df['month_Year'], format='%Y-%m-%d', errors = 'coerce')
#    )
#NOTE: I split the code above just for discussion on the SYNTAX with the team (it can be set back to one line after).
#NOTE2: the following between the "<>" failed ... because I need to convert to STRING ... to PARSE ... via the Python "str.split" function ... <HP_Split_DF =  Orig_HP_DateSeries.str.split('-')>
#NOTE3: ACTUALLY NO ... the following is "just a trim" ... we DON'T WANT THAT - Eventually we want "lstrip" via URL => https://www.tutsmake.com/python-trim-left-or-right-string/
#NOTE3: NEW APPROACH  via URL => https://www.statology.org/pandas-convert-datetime-to-string/

In [ ]:
#This cell is executing Step #3 & #4 above

Manual_Merge_HP_Date =  pd.concat([HP_Year_Row,HP_Month_Row,HP_Day_Row], axis=1)
#Manual_Merge_HP_Date
Manual_Merge_HP_Date.columns.values[0] = "HousePrice_Year"
Manual_Merge_HP_Date.columns.values[1] = "HousePrice_Month"
Manual_Merge_HP_Date.columns.values[2] = "HousePrice_Day"

Manual_Merge_HP_Date

#EXAMPLE Code Reference from above.
#Manual_Merge_date = pd.concat([Row_Year,Row_Month], axis=1) #axis=1 is using "column 1" to merge.
#Manual_Merge_date
#Rename Options presented via => https://www.geeksforgeeks.org/rename-specific-columns-in-pandas/
#Manual_Merge_date.columns.values[0] = "Year"
#Manual_Merge_date.columns.values[1] = "Month"
#Manual_Merge_date
#BEST PRACTICE MIGHT BE TO RENAME RIGHT AWAY "HERE" ... to save coding time later.

In [ ]:
#This cell is merging back into the Original .csv "user data" and also removing columns no longer needed.
#Purpose = to prepare to merge to the "master Analysis data sheet" on a city by city basis.

Updated_HP_by_Year_DF = pd.concat([Housing_Prices_by_year_df,Manual_Merge_HP_Date], axis=1,join='outer')
#Updated_HP_by_Year_DF

Van_Analysis_HP_by_Year_DF = Updated_HP_by_Year_DF[["HousePrice_Year","HousePrice_Month","HousePrice_Day","Vancouver"]] #,"Montreal","Toronto"]]
Van_Analysis_HP_by_Year_DF


#Step1>EXAMPLE CODE from earlier in the Notebook:
#GeoUpdated_DF = pd.concat([reduced_main_df,Manual_Merge_Geo],axis=1,join='outer') # ,join_axes=None,ignore_index=False)
#GeoUpdated_DF
#Step2>EXAMPLE CODE from earlier:
#Analysis_DF = DateUpdated_DF[["Year","Month","City_Region","Province","NewHousePrice_Index_Type","UOM","VALUE","STATUS","DECIMALS"]]
#Analysis_DF

In [ ]:
#Same as above for Montreal. KEY NOTE not cleaning this data (IE: "nulls/NaN") until we get into the Master Data Set.

Mtl_Analysis_HP_by_Year_DF = Updated_HP_by_Year_DF[["HousePrice_Year","HousePrice_Month","HousePrice_Day","Montreal"]] #,"Vancouver","Montreal","Toronto"]]
Mtl_Analysis_HP_by_Year_DF

In [ ]:
#Same as above for Montreal. KEY NOTE not cleaning this data (IE: "nulls/NaN") until we get into the Master Data Set.

Tor_Analysis_HP_by_Year_DF = Updated_HP_by_Year_DF[["HousePrice_Year","HousePrice_Month","HousePrice_Day","Toronto"]] #,"Vancouver","Montreal","Toronto"]]
Tor_Analysis_HP_by_Year_DF

In [ ]:
#Creating a test cell for the "if + elif" row ... as For (?) LOOP to populate a data frame. I'm not 100% certain the cells above are executing 100% of everything we need.



In [ ]:
#SET UP the basepoints for the calculation with historical prices ...
#Create VARs to enable them to run against the cleaned data set ...

#Determine how to complete the analysis ... and fit into how they can be placed into slides.

#Confirm the types of "plots" ... we're confident in to speak to the story of the data analysis we're able to produce.
#Remember to comment that our $ costs are "approximations" ... due to needing the baseprice assumptions.

In [ ]:
#Slides Notes:
#Maria is testing
